In [23]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

In [24]:
fit_data_xml = Path('converted_data/fit_data_xml.csv')

In [25]:
df = pd.read_csv(fit_data_xml)

In [26]:
df.head()

In [27]:
df.columns

In [28]:
df.iloc[0:4]

In [29]:
df.loc[df['activity'] == "Running"]

In [30]:
df.describe()

In [31]:
df.sort_values(['calories'], ascending=False)

In [32]:
dates = df['date']

In [33]:
new_date = []
for date in dates:
    str_date = (date.replace('T', ' ').split('.')[0])
    
    convert_date = datetime.strptime(str_date, "%Y-%m-%d %H:%M:%S")
    new_date.append(convert_date)

In [34]:
df['date'] = new_date

In [35]:
df.head()